# Démo Agent

### Environnement

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Imports

In [ ]:
import os
import uuid
from langchain_core.messages import HumanMessage

# infrastructure
from sources.agent_multi_tools.infrastructure.chat_history.in_memory_chat_history_handler import InMemoryChatHistoryHandler
from sources.agent_multi_tools.infrastructure.database.postgres_databse_handler import PostgresDatabaseHandler
from sources.agent_multi_tools.infrastructure.embeddings.ollama_embedding_handler import OllamaEmbeddingHandler
from sources.agent_multi_tools.infrastructure.llm.ollama_llm_handler import OllamaLLMHandler
from sources.agent_multi_tools.infrastructure.weather_api.open_meteo_api_handler import OpenMeteoApiHandler

# agent
from sources.agent_multi_tools.domain.services.agent.agent_state import AgentState
from sources.agent_multi_tools.domain.services.agent.agent_state_graph import AgentStateGraph

# tools
from sources.agent_multi_tools.domain.services.tools.conversation import Conversation
from sources.agent_multi_tools.domain.services.tools.rag import Rag
from sources.agent_multi_tools.domain.services.tools.text_to_sql import TextToSql
from sources.agent_multi_tools.domain.services.tools.weather import Weather

# config
from sources.agent_multi_tools.config.config_sql import ConfigSanrioData
from sources.agent_multi_tools.config.config_tools import ConfigTools

In [3]:
# Handler
chat_history_handler = InMemoryChatHistoryHandler()
embeddings_handler = OllamaEmbeddingHandler()
database_handler = PostgresDatabaseHandler(
    host = os.getenv("POSTGRES_HOST"),
    port = os.getenv("POSTGRES_PORT"),
    database = os.getenv("POSTGRES_DB"),
    username = os.getenv("POSTGRES_USER"),
    password = os.getenv("POSTGRES_PASSWORD"),
    embedding_handler=embeddings_handler,
)
llm_handler = OllamaLLMHandler(
    base_url=os.getenv("OLLAMA_URL"),
    model=os.getenv("OLLAMA_MODEL"),
)
weather_api_handler = OpenMeteoApiHandler()

In [4]:
# Tools
text_to_sql_tool = TextToSql(
    database_handler=database_handler,
    llm_handler=llm_handler,
    chat_history_handler=chat_history_handler,
    tables=[ConfigSanrioData]
)

rag_tool = Rag(
    database_handler=database_handler,
    llm_handler=llm_handler,
    chat_history_handler=chat_history_handler,
)

conversation_tool = Conversation(
    database_handler=database_handler,
    llm_handler=llm_handler,
    chat_history_handler=chat_history_handler,
)

weather_tool = Weather(
    weather_api_handler=weather_api_handler,
    llm_handler=llm_handler,
    chat_history_handler=chat_history_handler
)

tools = {
    ConfigTools.conversation: conversation_tool,
    ConfigTools.rag: rag_tool,
    ConfigTools.text_to_sql: text_to_sql_tool,
    ConfigTools.weather_api: weather_tool,
}

In [5]:
agent_state_graph = AgentStateGraph(llm_handler=llm_handler, chat_history_handler=chat_history_handler, tools=tools).compile_workflow(state=AgentState, memory=True)

In [ ]:
config = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "session_id": "f9gwxxfnv31" 
    }
}

In [ ]:
def _print_event(event: dict, _printed: set):
    messages = event.get("messages", [])
    if not messages:
        return
        
    message = messages[-1]
    
    msg_id = id(message)
    if msg_id in _printed:
        return
    _printed.add(msg_id)
    
    msg_type = type(message).__name__
    
    if msg_type == "HumanMessage":
        print(f"❓ Question: {message.content}")
        
    elif msg_type == "AIMessage":
        if hasattr(message, 'tool_calls') and message.tool_calls:
            tool_name = message.tool_calls[0]['name']
            print(f"🔧 Outil: {tool_name}")
        else:
            print(f"🤖 Réponse: {message.content}")
            
    elif msg_type == "ToolMessage":
        content = message.content
        if isinstance(content, dict) and 'answer' in content:
            print(f"📊 Résultat: {content['answer']}")
        else:
            print(f"📊 Résultat: {str(content)}")

In [ ]:
# question = "Qu'est ce que Valkyria ?"
question = "Quel temps fera-t-il demain ?"
# question = "J'aimerais connaitre l'age et la ville de Nyanmi dans Sanrio."
# question = "Peux tu ajouter l'id ?"
# question = "Comment vas tu ?"

_printed = set()

events = agent_state_graph.stream(
    {"messages": [HumanMessage(content=question)]}, 
    config, 
    stream_mode="values"
)
for event in events:
    _print_event(event, _printed)

In [ ]:
from sources.agent_multi_tools.infrastructure.interface.chat_handler import ChatHandler

question = "Quel temps fera-t-il demain ?"

chat = ChatHandler(agent_state_graph, chat_history_handler)
chat.handle_message(question, "f9gwxxfnv31")